# SPARQL Update, RDF Datasets

## Preparations

In [475]:
# Install required packages
# !pip install -q rdflib     # commented to avoid re-install with every re-run

### Imports and Functions 

We are re-using the sparql_select function. 

In [476]:
# Imports
import pandas as pd
import rdflib

# Convenient Functions
def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = []                             # a list of dictionaries, as intermediate format to construct the pandas DataFrame
  for result in results:                # iterate over the result set of the query, a result is an instance of rdflib.query.ResultRow
    row = {}                            #     create a dictionary to hold a single row of the result
    for var in results.vars:            #     iterate over the variables of the SPARQLResult to add a dictionary entry for each variable
      if (isinstance(result[var],URIRef) and use_prefixes):
        row[var] = result[var].n3(graph.namespace_manager)   # use namespace prefixes to shorten URIs
      else:
        row[var] = result[var]                  
    rows.append(row)                    #     add the dictionary (row) to the list 
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      


## SPARQL Update on single RDF Graphs

### Insert Data

In [477]:
g = Graph().parse(format="turtle",data="""
  @prefix : <http://example.org/> .""")

g.update("""
INSERT DATA { 
  :jane  a :Person; 
    :gender "female"@en; :age 22;
    :friend :mary, :bob, :bill;
    :loves :bill.
  :mary  a :Person; 
    :gender "female"; :age 22;
    :friend :bob;
    :loves :bill.
  :bob  a :Person;  
    :age 26; 
    :loves :jane.
};

INSERT DATA { 
  :mary :age 24.
  :bob a :Person; 
    :age 28.
  :bill  a :Person; 
    :gender "male";
    :friend :mary, :jane. 
}
""")

print(g.serialize(format="turtle"))

@prefix : <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:bob a :Person ;
    :age 26,
        28 ;
    :loves :jane .

:jane a :Person ;
    :age 22 ;
    :friend :bill,
        :bob,
        :mary ;
    :gender "female"@en ;
    :loves :bill .

:mary a :Person ;
    :age 22,
        24 ;
    :friend :bob ;
    :gender "female" ;
    :loves :bill .

:bill a :Person ;
    :friend :jane,
        :mary ;
    :gender "male" .




### Delete Data

In [478]:
g.update("""
DELETE DATA 
  { :mary :age 24.
    :bob :age 28.
    :bob :age 43.
  }""")

print(g.serialize(format="turtle"))

@prefix : <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:bob a :Person ;
    :age 26 ;
    :loves :jane .

:jane a :Person ;
    :age 22 ;
    :friend :bill,
        :bob,
        :mary ;
    :gender "female"@en ;
    :loves :bill .

:mary a :Person ;
    :age 22 ;
    :friend :bob ;
    :gender "female" ;
    :loves :bill .

:bill a :Person ;
    :friend :jane,
        :mary ;
    :gender "male" .




### Delete/Insert

In [479]:
g.update("""
DELETE {?p :age ?age_old}
INSERT {?p :age ?age_new}
WHERE 
  { ?p a :Person. 
    ?p :age ?age_old.
    BIND(?age_old + 1 AS ?age_new)	
  }""")

print(g.serialize(format="turtle"))

@prefix : <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:bob a :Person ;
    :age 27 ;
    :loves :jane .

:jane a :Person ;
    :age 23 ;
    :friend :bill,
        :bob,
        :mary ;
    :gender "female"@en ;
    :loves :bill .

:mary a :Person ;
    :age 23 ;
    :friend :bob ;
    :gender "female" ;
    :loves :bill .

:bill a :Person ;
    :friend :jane,
        :mary ;
    :gender "male" .




### INSERT with Subquery in WHERE clause

In [480]:
g.update("""
INSERT {?p :nrOfFriends ?nr}
WHERE 
  { SELECT ?p (COUNT(?f) AS ?nr)
    WHERE 
      { ?p a :Person.
        ?p :friend ?f.
      }
    GROUP BY ?p  	
  }""")

print(g.serialize(format="turtle"))

@prefix : <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:bob a :Person ;
    :age 27 ;
    :loves :jane .

:jane a :Person ;
    :age 23 ;
    :friend :bill,
        :bob,
        :mary ;
    :gender "female"@en ;
    :loves :bill ;
    :nrOfFriends 3 .

:mary a :Person ;
    :age 23 ;
    :friend :bob ;
    :gender "female" ;
    :loves :bill ;
    :nrOfFriends 1 .

:bill a :Person ;
    :friend :jane,
        :mary ;
    :gender "male" ;
    :nrOfFriends 2 .




### Update and Re-Calculation

To maintain derived materialized information we have to delete previously materialized derived statements and insert the newly derived statements.

In [481]:
g.update("""
DELETE DATA 
  { :jane :friend :bill };
 

DELETE WHERE {?p :nrOfFriends ?nr};

INSERT {?p :nrOfFriends ?nr}
WHERE 
  { SELECT ?p (COUNT(?f) AS ?nr)
    WHERE 
      { ?p a :Person.
        ?p :friend ?f.
      }
    GROUP BY ?p  	
  }
""")

print(g.serialize(format="turtle"))

@prefix : <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:bill a :Person ;
    :friend :jane,
        :mary ;
    :gender "male" ;
    :nrOfFriends 2 .

:bob a :Person ;
    :age 27 ;
    :loves :jane .

:jane a :Person ;
    :age 23 ;
    :friend :bob,
        :mary ;
    :gender "female"@en ;
    :loves :bill ;
    :nrOfFriends 2 .

:mary a :Person ;
    :age 23 ;
    :friend :bob ;
    :gender "female" ;
    :loves :bill ;
    :nrOfFriends 1 .




## Querying RDF Datasets

An RDF dataset is a default graph and a set of default graphs. 


### Insert Data

rdflib's support of datasets, especially regarding the default graph, seems not to be fully standard conformant. It seems that we have to parse in the default graph separately. In the Trig output, the default graph is given in curly brackets.

In [482]:
ds = rdflib.Dataset()
ds.default_context.parse(format="turtle", data="""
@prefix : <http://example.org/> .

:Jane a :Person;
  :owns :JanesGraph.

:Mary a :Person;
  :owns :MarysGraph.
  
:Bill a :Person;
  :owns :BillsGraph.
""")

ds.update("""
INSERT DATA {  
  GRAPH :JanesGraph { 
    :Jane :likes :Mary.
    :Bill :likes :Jane, :Mary.
  }

  GRAPH :MarysGraph {
    :Jane :likes :Mary.
    :Bill :likes :Jane.
    :Mary :likes :Jane.
  }

  GRAPH :BillsGraph {
    :Jane :likes :Mary, :Bill.
    :Bill :likes :Mary, :Jane.
  } 
}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix ns1: <urn:x-rdflib:> .

{
    :Bill a :Person ;
        :owns :BillsGraph .

    :Mary a :Person ;
        :owns :MarysGraph .

    :Jane a :Person ;
        :owns :JanesGraph .
}

:JanesGraph {
    :Bill :likes :Jane,
            :Mary .

    :Jane :likes :Mary .
}

:BillsGraph {
    :Bill :likes :Jane,
            :Mary .

    :Jane :likes :Bill,
            :Mary .
}

:MarysGraph {
    :Bill :likes :Jane .

    :Mary :likes :Jane .

    :Jane :likes :Mary .
}




### Querying all Triples and Quadruples in a Dataset

In [483]:
df = sparql_select(ds,"""
SELECT ?s ?p ?o ?g
WHERE {  
  { ?s ?p ?o }
  UNION
  { GRAPH ?g {?s ?p ?o}
   } 
}
ORDER BY ?g ?s ?p ?o
""")
df

,s,p,o,g
0,:Bill,:owns,:BillsGraph,None
1,:Bill,rdf:type,:Person,None
2,:Jane,:owns,:JanesGraph,None
3,:Jane,rdf:type,:Person,None
4,:Mary,:owns,:MarysGraph,None
5,:Mary,rdf:type,:Person,None
6,:Bill,:likes,:Jane,:BillsGraph
7,:Bill,:likes,:Mary,:BillsGraph
8,:Jane,:likes,:Bill,:BillsGraph
9,:Jane,:likes,:Mary,:BillsGraph


### Querying a specific Named Graph

In [484]:
df = sparql_select(ds,"""
SELECT *
WHERE 
  {  GRAPH :JanesGraph 
         {:Bill :likes ?o}
      
  }
""")
df

,o
0,:Jane
1,:Mary


### Intersection/Join of Named Graphs

In [485]:
df = sparql_select(ds,"""
SELECT DISTINCT *
WHERE 
  {  GRAPH :JanesGraph 
         {?s ?p ?o}
     GRAPH :MarysGraph 
         {?s ?p ?o}
  }
""")
df

,s,p,o
0,:Jane,:likes,:Mary
1,:Bill,:likes,:Jane


### Union of Named Graphs

In [486]:
df = sparql_select(ds,"""
SELECT DISTINCT *
WHERE 
  {  { GRAPH :JanesGraph 
         {?s ?p ?o}
     }
     UNION 
     { GRAPH :MarysGraph 
         {?s ?p ?o}
     }
  } 
""")
df

,s,p,o
0,:Jane,:likes,:Mary
1,:Bill,:likes,:Jane
2,:Bill,:likes,:Mary
3,:Mary,:likes,:Jane


### Querying graphs with a description fulfilling a given condition

In [487]:
df = sparql_select(ds,"""
SELECT *
WHERE 
  { :Jane :owns ?g.
    GRAPH ?g 
         {:Bill :likes ?o}
      
  }
""")
df

,g,o
0,:JanesGraph,:Jane
1,:JanesGraph,:Mary


#### Union of all person-owned graphs

In [488]:
df = sparql_select(ds,"""
SELECT DISTINCT ?s ?p ?o
WHERE 
  {  [] a :Person; :owns ?g.
   
     GRAPH ?g 
         {?s ?p ?o}
  }
""")
df

,s,p,o
0,:Jane,:likes,:Mary
1,:Bill,:likes,:Jane
2,:Bill,:likes,:Mary
3,:Mary,:likes,:Jane
4,:Jane,:likes,:Bill


### Correlating inner and outer queries

In [489]:
df = sparql_select(ds,"""
SELECT ?s ?p ?o
WHERE 
  {  ?s a :Person; 
        :owns ?g.
     GRAPH ?g 
         {?s ?p ?o}
  }
""")
df

,s,p,o
0,:Jane,:likes,:Mary
1,:Mary,:likes,:Jane
2,:Bill,:likes,:Jane
3,:Bill,:likes,:Mary


## Update the Dataset

Note: I haven't found out how to update, in rdflib, the default graph of the dataset using SPARQL Update.  

### Insert Data into a Named Graph

In [490]:
ds.update("""
INSERT DATA { 
  GRAPH :MarysGraph {
    :Mary :likes :Bill.
  }
}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix ns1: <urn:x-rdflib:> .

{
    :Bill a :Person ;
        :owns :BillsGraph .

    :Mary a :Person ;
        :owns :MarysGraph .

    :Jane a :Person ;
        :owns :JanesGraph .
}

:JanesGraph {
    :Bill :likes :Jane,
            :Mary .

    :Jane :likes :Mary .
}

:BillsGraph {
    :Bill :likes :Jane,
            :Mary .

    :Jane :likes :Bill,
            :Mary .
}

:MarysGraph {
    :Bill :likes :Jane .

    :Mary :likes :Bill,
            :Jane .

    :Jane :likes :Mary .
}




### Insert Data into a **new** Named Graph

In [491]:
ds.update("""
INSERT DATA { 
  GRAPH :NewGraph {
    :Mary :likes :Bill.
  }
}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix ns1: <urn:x-rdflib:> .

{
    :Bill a :Person ;
        :owns :BillsGraph .

    :Jane a :Person ;
        :owns :JanesGraph .

    :Mary a :Person ;
        :owns :MarysGraph .
}

:MarysGraph {
    :Bill :likes :Jane .

    :Mary :likes :Bill,
            :Jane .

    :Jane :likes :Mary .
}

:JanesGraph {
    :Bill :likes :Jane,
            :Mary .

    :Jane :likes :Mary .
}

:NewGraph {
    :Mary :likes :Bill .
}

:BillsGraph {
    :Bill :likes :Jane,
            :Mary .

    :Jane :likes :Bill,
            :Mary .
}




### Cut and Paste

Delete some triples from different graphs and insert them into another graph. 

In [492]:
ds.update("""
DELETE WHERE { GRAPH :NewGraph { ?s ?p ?o. } }
""")

ds.update("""
DELETE {  
  GRAPH ?g 
    { :Bill :likes ?o. } } 
INSERT { 
  GRAPH :BGraph 
    { :Bill :likes ?o. } }
WHERE {
  GRAPH ?g 
    { :Bill :likes ?o. } }
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix ns1: <urn:x-rdflib:> .

{
    :Bill a :Person ;
        :owns :BillsGraph .

    :Jane a :Person ;
        :owns :JanesGraph .

    :Mary a :Person ;
        :owns :MarysGraph .
}

:MarysGraph {
    :Jane :likes :Mary .

    :Mary :likes :Bill,
            :Jane .
}

:JanesGraph {
    :Jane :likes :Mary .
}

:BillsGraph {
    :Jane :likes :Bill,
            :Mary .
}

:BGraph {
    :Bill :likes :Jane,
            :Mary .
}


